In [86]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
%matplotlib inline

I want to make a model that can predict a cancer diagnosis.

In [61]:
cancer = pd.read_csv('C:\Code\Data\\breast_cancer.csv')
display(cancer.head())
print(cancer.shape)

,1000025,5,1,1.1,1.2,2,1.3,3,1.4,1.5,2.1
0,1002945,5,4,4,5,7,10,3,2,1,2
1,1015425,3,1,1,1,2,2,3,1,1,2
2,1016277,6,8,8,1,3,4,3,7,1,2
3,1017023,4,1,1,3,2,1,3,1,1,2
4,1017122,8,10,10,8,7,10,9,7,1,4


(698, 11)


In [63]:
for i in range(0,len(cancer.columns)):
    try:
        cancer.iloc[:,i].astype(float)
    except:
        print(cancer.columns[i])

1.3


In [64]:
cancer = cancer.drop('1.3',axis=1)

In [65]:
newrow = cancer.columns


In [67]:
cancer.columns = ['IDNum','Thickness','Size_Uniformity','Shape_Uniformity','Adhesion',
                  'Epithelial_Size','Chromatin','Normal_Nucleoli','Mitoses','Class']

In [76]:
newrow = pd.Series(newrow)
newrow = newrow.astype(float)
newrow.index = ['IDNum','Thickness','Size_Uniformity','Shape_Uniformity','Adhesion',
                  'Epithelial_Size','Chromatin','Normal_Nucleoli','Mitoses','Class']
cancer = cancer.append(newrow, ignore_index=True)

In [79]:
display(cancer.head())

,IDNum,Thickness,Size_Uniformity,Shape_Uniformity,Adhesion,Epithelial_Size,Chromatin,Normal_Nucleoli,Mitoses,Class
0,1002945.0,5.0,4.0,4.0,5.0,7.0,3.0,2.0,1.0,0
1,1015425.0,3.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,0
2,1016277.0,6.0,8.0,8.0,1.0,3.0,3.0,7.0,1.0,0
3,1017023.0,4.0,1.0,1.0,3.0,2.0,3.0,1.0,1.0,0
4,1017122.0,8.0,10.0,10.0,8.0,7.0,9.0,7.0,1.0,1


In [78]:
cancer.Class = np.where(cancer.Class==4,1,0)

In [82]:
X = cancer.drop(['IDNum','Class'], axis=1)
Y = cancer.Class

## Logistic Regression

In [102]:
#Vanilla Log
lr = linear_model.LogisticRegression(C=1e10)
lfit = lr.fit(X, Y)

print('R² for the Vanilla Log:')
print(lfit.score(X, Y))

print('\nCrossValidation for Vanilla Log:')
print(cross_val_score(lr, X, Y))

lrparams = np.append(lfit.coef_, lfit.intercept_)
print('\nParameter estimates for the Vanilla Model:')
print(lrparams)

R² for the Vanilla Log:
0.9614285714285714

CrossValidation for Lasso:
[0.92735043 0.93991416 0.97854077]

Parameter estimates for the Vanilla Model:
[ 0.57702383 -0.01084091  0.56927257  0.31392001  0.13077332  0.5794412
  0.1232866   0.60627102 -9.950884  ]


## Ridge

In [103]:
grid = [.1, 1, 10,100,200,300,500,700, 1000, 10000] 
out = [] 
for c in grid: 
    lrr = linear_model.LogisticRegression(C=c) 
    lrr.fit(X, Y) 
    scores = cross_val_score(lrr, X, Y,) 
    out.append(scores.mean()) 
    bestc = grid[out.index(max(out))] 

lrr = linear_model.LogisticRegression(C=bestc) 
lrr.fit(X,Y)
print('The Model was fit with C = ',bestc)

print('\nR² for the Ridge model:')
print(lrr.score(X, Y))

print('\nCrossValidation for Rigde Log:')
print(cross_val_score(lrr, X, Y))

ridgeparams = np.append(lrr.coef_, lrr.intercept_)
print('\nParameter estimates for the Ridge Model:')
print(ridgeparams)


The Model was fit with C =  10

R² for the Ridge model:
0.9614285714285714

CrossValidation for Rigde Log:
[0.92307692 0.93991416 0.98283262]

Parameter estimates for the Ridge Model:
[ 0.51607307  0.02328681  0.53662857  0.2897733   0.10149863  0.52181515
  0.12628892  0.48769316 -9.08892262]


In [104]:
grid = [.01,.1, 1, 10,100,200,300,500,700, 1000, 10000] 
out = [] 
for c in grid: 
    lrl = linear_model.LogisticRegression(penalty='l1',C=c) 
    lrl.fit(X, Y) 
    scores = cross_val_score(lrl, X, Y) 
    out.append(scores.mean()) 
    bestc = grid[out.index(max(out))]

lrl = linear_model.LogisticRegression(penalty='l1',C=bestc) 
lrl.fit(X,Y)
print('The Model was fit with C = ',bestc)

print('\nR² for the Lasso model:')
print(lrl.score(X, Y))

print('\nCrossValidation for Lasso:')
print(cross_val_score(lrl, X, Y))

lassparams = np.append(lrl.coef_, lrl.intercept_)
print('\nParameter estimates for the Lasso Model:')
print(lassparams)

The Model was fit with C =  0.1

R² for the Lasso model:
0.9614285714285714

CrossValidation for Lasso:
[0.92307692 0.94420601 0.98712446]

Parameter estimates for the Lasso Model:
[ 0.26033389  0.19653691  0.36004226  0.18038346  0.          0.23725207
  0.13536456  0.         -5.29515286]


http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Original%29